In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/carl/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
dataset_path = "/home/carl/data_sawyer/dclose_11_25/demos_10d_g53.hdf5"

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())

lengths=[]
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  81
Max length:  636
Min length:  43
Mean length:  128.65432098765433


In [4]:
del f['mask']['good']
f['mask'].keys()

<KeysViewHDF5 ['bad', 'g50b10', 'g50b20', 'g50b5']>

In [5]:
demos_good=np.array(demos[:53], dtype='S8')
demos_good.shape

(53,)

In [6]:
hdf5_path=dataset_path

filter_keys=demos_good
filter_name="good"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [7]:
f['mask'].keys()

<KeysViewHDF5 ['bad', 'g50b10', 'g50b20', 'g50b5', 'good']>

In [6]:
max_id=0
for d in demos:
    id=int(d.split('_')[1])
    if id>max_id:
        max_id=id
next_id=max_id+1
next_id

53

In [10]:
f.close()

In [7]:
dataset_path2 = "/home/carl/data_sawyer/dclose/demos_10d_b28.hdf5"

f2 = h5py.File(dataset_path2, "r")
demos2 = list(f2["data"].keys())

lengths=[]
for demo_name in demos2:
    demo=f2['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos2))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  28
Max length:  636
Min length:  87
Mean length:  268.57142857142856


In [8]:
keys_to_copy=demos2

In [9]:
i=next_id
for demo_name in tqdm.tqdm(keys_to_copy):
    b_demo=f2['data'][demo_name]
    num_samples=b_demo.attrs['num_samples']

    bdemo_name="demo_"+str(i)
    i = i+1

    f["data"].create_group(bdemo_name)


    ks=['actions', 'times']
    for k in ks:
        kdata = b_demo[k]
        f["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =b_demo['obs'].keys()

    for k in obs_keys:
        kdata = b_demo['obs'][k]
        f["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)

    f["data"][bdemo_name].attrs["num_samples"] = num_samples
 
f2.close()

100%|██████████| 28/28 [00:00<00:00, 95.06it/s]


In [10]:
# create a group for the new data.
i=next_id
keys=[]
for demo_name in tqdm.tqdm(keys_to_copy):
    bdemo_name="demo_"+str(i)
    keys.append(bdemo_name)
    i = i+1

keys=np.array(keys, dtype='S8')

100%|██████████| 28/28 [00:00<00:00, 343393.31it/s]


In [12]:
filter_keys=keys
filter_name="bad"
filter_lengths = create_hdf5_filter_key(hdf5_path=dataset_path, demo_keys=filter_keys, key_name=filter_name)

In [13]:
# verify length
demos = list(f["data"].keys())
len(demos)

81

In [14]:
good=[demo.decode('utf-8') for demo in f['mask']['good'] ] 
bad=[demo.decode('utf-8') for demo in f['mask']['bad'] ]  
len(good) , len(bad) 

(53, 28)

In [16]:
gb1=good[:50] + bad[:5]
gb2=good[:50] + bad[:10]
gb3=good[:50] + bad[:20]

gb1=np.array(gb1, dtype='S8')
gb2=np.array(gb2, dtype='S8')
gb3=np.array(gb3, dtype='S8')

gb1.shape, gb2.shape, gb3.shape

((55,), (60,), (70,))

In [17]:
filter_keys=gb1
filter_name='g50b5'
filter_lengths = create_hdf5_filter_key(hdf5_path=dataset_path, demo_keys=filter_keys, key_name=filter_name)

filter_keys=gb2
filter_name='g50b10'
filter_lengths = create_hdf5_filter_key(hdf5_path=dataset_path, demo_keys=filter_keys, key_name=filter_name)

filter_keys=gb3
filter_name='g50b20'
filter_lengths = create_hdf5_filter_key(hdf5_path=dataset_path, demo_keys=filter_keys, key_name=filter_name)

In [18]:
f.close()

### verify

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
print('dataset: ', dataset_path)
for k in f['mask'].keys():
    ks=f['mask'][k]
    print(k, ks)
f.close()

dataset:  /home/carl/data_sawyer/dclose_11_25/demos_10d_g53.hdf5
bad <HDF5 dataset "bad": shape (28,), type "|S8">
g50b10 <HDF5 dataset "g50b10": shape (60,), type "|S8">
g50b20 <HDF5 dataset "g50b20": shape (70,), type "|S8">
g50b5 <HDF5 dataset "g50b5": shape (55,), type "|S8">
good <HDF5 dataset "good": shape (53,), type "|S8">
